# HypergraphPercol Colab build

This notebook reproduces the multi-stage Docker build pipeline inside a Google Colab runtime so that the `HypergraphPercol` package and its CGAL helpers are available directly from a Colab session.

> **Execution order**
> Run the cells sequentially from top to bottom in a fresh Colab runtime. The build relies on system packages, so restarting midway may require rerunning the earlier cells.

## 1. Install system dependencies

The Dockerfile installs a series of Ubuntu packages that provide CGAL, Boost, Eigen and a modern build toolchain. We replicate the same setup here.

In [1]:
%%bash
set -euo pipefail
apt-get update
DEBIAN_FRONTEND=noninteractive apt-get install -y     build-essential     cmake     git     libtbb-dev     libcgal-dev     libboost-all-dev     libeigen3-dev


Ign:1 https://mise.jdx.dev/deb stable InRelease


Hit:3 http://archive.ubuntu.com/ubuntu noble InRelease


Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease


Hit:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease


Hit:6 http://archive.ubuntu.com/ubuntu noble-backports InRelease


...

build-essential is already the newest version (12.10ubuntu1).
cmake is already the newest version (3

.28.3-1build7).
git is already the newest version (1:2.43.0-1ubuntu7.3).
libtbb-dev is already the n

ewest version (2021.11.0-2ubuntu2).
libcgal-dev is already the newest version (5.6-1build3).
libboos

t-all-dev is already the newest version (1.83.0.1ubuntu2).
libeigen3-dev is already the newest versi

on (3.4.0-4build0.1).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


## 2. Upgrade `pip` and core Python build tooling

The compiled extensions require an up-to-date Python build stack along with Cython and NumPy 1.26.x (force reinstalled to avoid mixed ABI issues on Colab). We also pre-install `cmake` through `pip` to match the Docker workflow.

In [2]:
%%bash
set -euo pipefail
python3 -m pip install --upgrade pip setuptools wheel "Cython>=3.0" cmake jedi
python3 -m pip install --upgrade "numpy>=2,<3"


.3)


ges (80.9.0)


0.45.1)


ages (3.1.6)


4.1.2)


...

on if you know what you are doing and want to suppress this warning.


ages (2.3.4)


our with the system package manager, possibly rendering your system unusable. It is recommended to u

se a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action opti

on if you know what you are doing and want to suppress this warning.


## 3. Clone the required repositories

We fetch both the main HypergraphPercol sources and the `cyminiball` dependency at the same revisions used by the Docker build.

In [3]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
mkdir -p "${WORKDIR}"
cd "${WORKDIR}"
if [ -d HypergraphPercol ]; then
    git -C HypergraphPercol pull --ff-only
else
    git clone https://github.com/Ludwig-H/HypergraphPercol.git
fi
if [ -d cyminiball ]; then
    git -C cyminiball pull --ff-only
else
    git clone https://github.com/Ludwig-H/cyminiball.git
fi


Already up to date.


Already up to date.


## 4. Build and install `cyminiball`

The Docker image creates a wheel from source and installs it without build isolation. We mirror that approach so the same binary is present inside Colab.

In [4]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
mkdir -p "${WORKDIR}/wheels"
cd "${WORKDIR}/cyminiball"
python3 -m pip wheel --no-build-isolation --wheel-dir="${WORKDIR}/wheels" .
python3 -m pip install --force-reinstall --no-index --find-links="${WORKDIR}/wheels" cyminiball


Processing /workspace/hgp_colab/cyminiball


  Preparing metadata (pyproject.toml): started


  Preparing metadata (pyproject.toml): finished with status 'done'


  File was already downloaded /workspace/hgp_colab/wheels/numpy-2.3.4-cp312-cp312-manylinux_2_27_x86

_64.manylinux_2_28_x86_64.whl


...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

━━━━━━━━━━━ 2/2 [cyminiball]


our with the system package manager, possibly rendering your system unusable. It is recommended to u

se a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action opti

on if you know what you are doing and want to suppress this warning.


## 5. Download the CGAL helper projects

The helper script clones the six CGAL-based executables required by HypergraphPercol.

In [5]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HypergraphPercol"
python3 scripts/setup_cgal.py


[skip] EdgesCGALWeightedDelaunay3D already exists
[skip] EdgesCGALWeightedDelaunay2D already exists


[skip] EdgesCGALWeightedDelaunayND already exists
[skip] EdgesCGALDelaunay3D already exists
[skip] E

dgesCGALDelaunay2D already exists
[skip] EdgesCGALDelaunayND already exists


## 6. Patch (if necessary) and compile the CGAL executables

The 3D helpers require an explicit pthread linkage, just like in the Dockerfile. Each project is then configured and built in Release mode.

In [6]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HypergraphPercol/CGALDelaunay"

projects=(
    EdgesCGALDelaunay2D
    EdgesCGALDelaunay3D
    EdgesCGALDelaunayND
    EdgesCGALWeightedDelaunay2D
    EdgesCGALWeightedDelaunay3D
)

for project in "${projects[@]}"; do
    cmake -S "${project}" -B "${project}/build" -DCMAKE_BUILD_TYPE=Release
    cmake --build "${project}/build" --config Release
    cmake --install "${project}/build" --prefix "${WORKDIR}/HypergraphPercol"
done


-- Using header-only CGAL


-- Targeting Unix Makefiles
-- Using /usr/bin/c++ compiler.


CMake Warning (dev) at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGAL_SetupBoost.cmake:20 (find_pack

age):
  Policy CMP0167 is not set: The FindBoost module is removed.  Run "cmake
  --help-policy CMP0

167" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning

...

-- Generating done (0.0s)


-- Build files have been written to: /workspace/hgp_colab/HypergraphPercol/CGALDelaunay/EdgesCGALWei

ghtedDelaunay3D/build


[100%] Built target EdgesCGALWeightedDelaunay3D


-- Install configuration: "Release"


## 7. Install Python runtime dependencies

HypergraphPercol depends on scientific Python packages such as scikit-learn, HDBSCAN and GUDHI. Installing them upfront ensures that the later `pip install` step can reuse the locally built `cyminiball` wheel without attempting to rebuild it. Some of these wheels may opportunistically upgrade NumPy, so we immediately reinstall 1.26.4 afterwards to keep the ABI aligned with the compiled extensions.

In [7]:
%%bash
set -euo pipefail
python3 -m pip install --upgrade scikit-learn hdbscan gudhi joblib threadpoolctl
python3 -m pip install --upgrade "numpy>=2,<3"


kages (1.7.2)


 (0.8.40)


3.11.0)


(1.5.2)


ckages (3.6.0)


...

on if you know what you are doing and want to suppress this warning.


ages (2.3.4)


our with the system package manager, possibly rendering your system unusable. It is recommended to u

se a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action opti

on if you know what you are doing and want to suppress this warning.


## 8. Install HypergraphPercol from source

Finally, install the package so that it becomes importable inside the notebook runtime. Using `--no-deps` keeps the `cyminiball` wheel we built earlier instead of asking `pip` to recompile it.

In [8]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HypergraphPercol"
python3 -m pip install --no-deps --force-reinstall .


Processing /workspace/hgp_colab/HypergraphPercol


  Installing build dependencies: started


  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: 

started


  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.t

...

    Uninstalling hypergraphpercol-0.1.0:


      Successfully uninstalled hypergraphpercol-0.1.0


our with the system package manager, possibly rendering your system unusable. It is recommended to u

se a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action opti

on if you know what you are doing and want to suppress this warning.


## 9. Configure the runtime (optional) and validate the installation

The package defaults to `/content/HypergraphPercol/CGALDelaunay` when `cgal_root` is not provided. The following cell sets the environment variable explicitly and performs a simple clustering run to ensure everything is functional.

In [9]:
import os
import numpy as np

workdir = os.environ.get("HGP_WORKDIR", "/content")
repo_root = os.path.join(workdir, "HypergraphPercol")
os.environ["CGALDELAUNAY_ROOT"] = os.path.join(repo_root, "CGALDelaunay")

from hypergraphpercol import HypergraphPercol

rng = np.random.default_rng(0)
data = np.vstack([
    rng.normal(loc=-2.0, scale=0.4, size=(40, 3)),
    rng.normal(loc=2.0, scale=0.4, size=(40, 3)),
])
labels = HypergraphPercol(
    data,
    K=2,
    min_cluster_size=20,
    min_samples=10,
    metric="euclidean",
    complex_chosen="auto",
    expZ=2,
    precision="safe",
    verbeux=True,
    cgal_root=os.environ["CGALDELAUNAY_ROOT"],
)
print("Unique labels:", np.unique(labels))


[info] TBB enabled, using up to 1 threads
[info] Loaded 80 points
[info] Wrote 492 edges to /tmp/tmp5d_w0x8f/edges80.npy
[info] TBB enabled, using up to 1 threads
[info] Loaded 492 points
[info] Spatially sorted points prior to insertion
[info] Wrote 3243 edges to /tmp/tmpksh45e3f/edges492.npy
[info] TBB enabled, using up to 1 threads
[info] Loaded 2539 points
[info] Spatially sorted points prior to insertion
[info] Wrote 8215 edges to /tmp/tmpmeaovvui/edges2539.npy


orderk_delaunay k = 1
Computed weighted barycentres 492
orderk_delaunay k = 2
Computed weighted barycentres 2539
orderk_delaunay k = 3
Computed weighted barycentres 6014


[info] TBB enabled, using up to 1 threads
[info] Loaded 6014 points
[info] Spatially sorted points prior to insertion
[info] Wrote 14975 edges to /tmp/tmp7ae8r_iw/edges6014.npy


orderk_delaunay k = 4
Computed weighted barycentres 10377


[info] TBB enabled, using up to 1 threads
[info] Loaded 10377 points
[info] Spatially sorted points prior to insertion
[info] Wrote 23199 edges to /tmp/tmph4xu4l6r/edges10377.npy


...

N_CPU_dispo utilisés :  -1


2-simplices=4915680


Faces uniques: 1927 (compression 14747040→1927)


Arêtes uniques (u<v): 35254 (avant dédup 9831360)
MST faces: 1926 arêtes, composantes estimées: 1
[COMP-F:0] components=1, faces=1927, points=80, RSS=3.2 GiB
[COMP-F:1] comp 1/1 | faces=1927, edges=1926, Z_est≈60.2 KiB, RSS=3.2 GiB
[PRUNE] leaves=1927, classes=110, Z_pruned shape=(109, 4)
[COMP-F:2] comp 1/1 done | cumul points labellisés=78 | conflits=2 | RSS=3.2 GiB
Clusters finaux : 2 | bruit : 2 | RSS=3.2 GiB


Unique labels: [-1  0  1]
